In [3]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    'splitdataset48x48\\train',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    'splitdataset48x48\\val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

NameError: name 'ImageDataGenerator' is not defined

In [13]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'B', 'C', 'D', 'E']


In [18]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(5, activation='softmax'))

In [19]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [20]:
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [21]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
9/9 [==============================] - 157s 17s/step - loss: 1.4759 - accuracy: 0.3237 - val_loss: 1.0984 - val_accuracy: 0.6094
Epoch 2/10
9/9 [==============================] - 147s 16s/step - loss: 0.8272 - accuracy: 0.6549 - val_loss: 0.3734 - val_accuracy: 0.7930
Epoch 3/10
9/9 [==============================] - 149s 17s/step - loss: 0.4170 - accuracy: 0.8218 - val_loss: 0.1136 - val_accuracy: 0.9648
Epoch 4/10
9/9 [==============================] - 150s 17s/step - loss: 0.1576 - accuracy: 0.9422 - val_loss: 0.0232 - val_accuracy: 0.9883
Epoch 5/10
1/9 [==>...........................] - ETA: 53s - loss: 0.1349 - accuracy: 0.9583

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


score = model.evaluate(validation_generator)
print("Validation Accuracy:", score[1])  

In [ ]:
model_json = model.to_json()
with open("model.json",'w') as json_file:
    json_file.write(model_json)
model.save("model.h5")